### 0. 환경 세팅

In [3]:
import re
import requests
from krwordrank.word import KRWordRank
from kiwipiepy import Kiwi
from konlpy.tag import Hannanum
import gensim
import pandas as pd

In [4]:
def clean_doc(doc) :
    doc = re.sub(r'[^\w\s]', " ", doc)

    # 인코딩 깨진 경우
    doc = doc.replace("\xa0", " ") # 공백문자

    return doc

### 1. 데이터 불러오기

In [ ]:
url = 'https://raw.githubusercontent.com/soyeon-stat/Bigdata/refs/heads/master/INSTIZ_CONTENTS.json'
response = requests.get(url)
jsonData = response.json()
sentence = jsonData[0]['content'] # 예시데이터 : instiz

### 2. 문장 나누기

In [7]:
kiwi = Kiwi()
text_list = []
for sent in kiwi.split_into_sents(sentence) :
    _sent = clean_doc(sent.text)
    text_list.append(_sent)

### 3. 핵심어 추출

##### (1) 말뭉치 생성

In [8]:
han = Hannanum()
corpus = [clean_doc(x) for x in text_list]

# 명사로 토큰화
tokenized_corpus = [
    han.nouns(doc) for doc in corpus 
]

In [9]:
# 원본글
for x in corpus : 
    print(x)

마플 부정적인 언급이 있어요전부터 억까라고 아무리 말해도 정정도 안되는거 같고 가수 욕먹는게 너무 답답해서 직접 자료 만들어와봤어 
얼마전 초록글에서도 댓글에 반박할거있으면 제대로 설명가져와보라길래금발에 파란 염색 올라간거 그거 하나빼곤 다 반박해왔어
그것도 찾으면 금발 파란 포인트 머리한 돌 여럿 나오긴 나와 
같이 정리할까하다가 뺀 이유는 타돌 언급 최대한 안하고 싶어서도 있고나머지가 다 억까란걸 설명했으니 정말 겹치는게 생겼다해도 그건 우연일수도 있단걸 말하고 싶었어
이거 읽고도 채원이가 윈터를 따라했다고 계속 믿고 억까하고 싶은 사람들 있을거고 안 말릴게 
근데 소수라도 억까였구나 깨달아주는 사람이 있을것 같아서 정리했어
내가 하고 싶은 말은 저 빨간글씨 날짜적힌 짤 자체가 억까라는 것과 머리 스타일이 좀 겹치더라도 그게 누가 누굴 따라했단게 되지는 않는다는거야 
솔직히 아이돌들 스타일 다 비슷비슷 돌아가면서 하고 유행도 비슷하잖아 
왜 억지로 엮어가면서 누굴 따라쟁이로 만들어야 속이 풀리는지 모르겠어 
게다가 시작은 네이트판 정병인데아이돌들은 수많은 머리를 하고 거기서 서로 비슷한 느낌 찾는거 어려운일 아니야
제발 이 정리글을 읽고 억까 좀 멈춰주면 좋겠어 
자료는 억까있는 곳이면 퍼가도 되는데 가능하면 여기 지금 본문에 적은 이 설명도 같이 퍼가주면 좋겠어추천  34카카오톡 엑스


##### (2) 핵심어 추출 : KRWordRank

In [13]:
min_count = 2   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
wordrank_extractor = KRWordRank(min_count=min_count, max_length=max_length)
beta = 0.85    # PageRank의 decaying factor beta
max_iter = 20
keywords, rank, graph = wordrank_extractor.extract(corpus, beta, max_iter)

for word, score in keywords.items() :
    print(f'{word} : {score}')

누굴 : 2.7521570648948255
같이 : 2.220973199809757
자료 : 1.9432857034876556
하고 : 1.8444746617250285
머리 : 1.8332568786090562
싶은 : 1.7698755412497542
좋겠어 : 1.3373487417397945
퍼가 : 1.134679729921292
만들어 : 1.0873959177144814
스타일 : 1.0628044301717359
겹치 : 1.01481182411515
사람 : 1.000373630966405
파란 : 0.9883369821845991
억까 : 0.9506499616606991
비슷 : 0.8001183633898706
읽고 : 0.792807107776494
정리 : 0.7716618960026717
설명 : 0.7636441609262122
따라 : 0.6833157355968886
반박 : 0.4555938786508919
언급 : 0.4376544736921543


##### (3) 핵심어 추출 : TF-IDF

In [14]:
lexicon = gensim.corpora.Dictionary(tokenized_corpus)
tfidf = gensim.models.TfidfModel(dictionary=lexicon, normalize = True)

In [15]:
doc_tfidf = pd.DataFrame()
for doc in tokenized_corpus :
    vec = lexicon.doc2bow(doc)
    vec = tfidf[vec]
    a = pd.DataFrame(vec, columns = ['label', 'score']) # 데이터프레임 생성
    doc_tfidf = pd.concat([doc_tfidf, a])

doc_tfidf = doc_tfidf.groupby('label')[['score']].mean().sort_values('score', ascending=False)

In [16]:
lexicon_dict = pd.DataFrame([x for x in lexicon.items()], columns = ['label', 'name'])
lexicon_dict = lexicon_dict.set_index('label')
print(doc_tfidf.join(lexicon_dict))

          score  name
label                
78     0.811128   정리글
71     0.629950     속
70     0.629950  따라쟁이
69     0.470364    유행
66     0.470364  비슷비슷
...         ...   ...
9      0.240958    자료
59     0.240520    머리
7      0.225686    언급
30     0.224551    설명
3      0.170383     말

[87 rows x 2 columns]
